In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [2]:
# file = "sat-data.h5"
# sat_data = pd.read_hdf(file)
filex = "sat-data.npy"
filey = "sat-labels.npy"

sat_data = np.load( filex )
sat_labels = np.load( filey, allow_pickle=True )

In [3]:
## Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sat_data, sat_labels, test_size=0.2, random_state=42)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Softmax, Conv1D, MaxPooling1D

# Define the CNN model
model = Sequential()

# input_shape = (20, 6, 1)
# First convolutional layer (assuming 1D data)
model.add(Conv1D(filters=128, kernel_size=2, activation='relu'))

# Second convolutional layer
model.add(Conv1D(filters=128, kernel_size=2, activation='relu'))

# Max pooling layer
model.add(MaxPooling1D(pool_size=2))

# Flatten the output
model.add(Flatten())

# Fully connected layers
model.add(Dense(units=192, activation='relu'))
model.add(Dense(units=89, activation='relu'))

# Output layer with 3 units for maneuver classification (no maneuver, beginning of maneuver, during maneuver)
model.add(Dense(units=3, activation='softmax'))

# Output layer with 3 output units for classification, [1, 0, 0], [0, 1, 0], [0, 0, 1] sigmoid activation function

# Compile the model (loss function and optimizer might be different in the paper)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Implement 10-fold cross-validation (code not shown here, refer to Kera's documentation)
model.build(input_shape=sat_data.shape)

model.summary()
# Train the model (refer to Keras documentation for training with validation sets)
# model.fit(X_train, y_train, epochs=5)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (127, 4, 128)             1664      
                                                                 
 conv1d_1 (Conv1D)           (127, 3, 128)             32896     
                                                                 
 max_pooling1d (MaxPooling1  (127, 1, 128)             0         
 D)                                                              
                                                                 
 flatten (Flatten)           (127, 128)                0         
                                                                 
 dense (Dense)               (127, 192)                24768     
                                                                 
 dense_1 (Dense)             (127, 89)                 17177     
                                                        

In [5]:
sat_data = sat_data.astype('float32')
sat_labels = np.array([label[0] for label in sat_labels]).astype('int32')


In [6]:
sat_labels[0]

array([1, 0, 0], dtype=int32)

In [7]:

model.fit(sat_data, sat_labels, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
4/4 [==============================] - 0s 37ms/step - loss: 1.0238 - accuracy: 0.5446 - val_loss: 1.1959 - val_accuracy: 0.1154
Epoch 2/10
4/4 [==============================] - 0s 6ms/step - loss: 0.7980 - accuracy: 0.6733 - val_loss: 1.4277 - val_accuracy: 0.1154
Epoch 3/10
4/4 [==============================] - 0s 8ms/step - loss: 0.6391 - accuracy: 0.7129 - val_loss: 1.7837 - val_accuracy: 0.1538
Epoch 4/10
4/4 [==============================] - 0s 9ms/step - loss: 0.4883 - accuracy: 0.8020 - val_loss: 1.9048 - val_accuracy: 0.5385
Epoch 5/10
4/4 [==============================] - 0s 5ms/step - loss: 0.3511 - accuracy: 0.9208 - val_loss: 2.1457 - val_accuracy: 0.6538
Epoch 6/10
4/4 [==============================] - 0s 5ms/step - loss: 0.2447 - accuracy: 0.9703 - val_loss: 2.8288 - val_accuracy: 0.6538
Epoch 7/10
4/4 [==============================] - 0s 5ms/step - loss: 0.1525 - accuracy: 0.9703 - val_loss: 3.7791 - val_accuracy: 0.6538
Epoch 8/10
4/4 [=================

In [8]:
# Evaluate the model
# model.evaluate(X_test, y_test)
model.evaluate(sat_data, sat_labels)

# show trustworthiness of the model
# model.predict(X_test)
model.predict(sat_data[:1])
# np.argmax(model.predict(sat_data[:1]), axis=1)


1/1 [==============================] - 0s 50ms/step


array([[9.9999845e-01, 1.0599462e-08, 1.5503838e-06]], dtype=float32)